In [5]:
library(foreach)
library(doParallel)
library(Matching)
library(plyr)
library(tidyverse)
library(stringr)
library(magrittr)
library(caret)
library(itecv)
# detach("package:plyr", unload=TRUE)

In [6]:
loop_test = function(setup) {
    DGP = setup$DGP
    n_train = setup$n_train
    n_test = setup$n_test
    n_folds = setup$n_folds
    training_percent = setup$training_percent
    rep = setup$rep
    models = setup$models
    
    n = n_train + n_test
    training_percent 
    
    datas = setup_data(DGP$DGP, n_train, n_test, n_folds)
    estimates = datas %$% get_estimates(data, models, cv_index, test_index) 
    errors =  estimates %$% get_errors(cv_estimates, test_estimates, datas$aux_data) 
    true_selection_error = errors$true_selection_error %>%
        mutate(DGP=DGP$name, n_train=n_train, n_test=n_test, n_folds=n_folds, rep=rep)
    
    return(true_selection_error)
}

In [8]:
methods = list("ranger",
               "enet")
tune_grids = list(expand.grid(mtry = c(3,5,7),
                              splitrule="variance",
                              min.node.size=1),
                  expand.grid(fraction=0.5, 
                              lambda=exp(seq(-5,2,0.5))))
models = list(methods, tune_grids) %>%
    pmap(~list(method=.x, tune_grid=.y))

simulation_params = list(DGP=powers_DGPs() %>% 
                             imap(~list(name=.y, DGP=.x)),
                         n_test = 2000,
                         n_train = 1000,
                         n_folds=c(2,5,10),
                         rep = 1:20) %>%
cross
setups = simulation_params %>%
    map(~c(.,"models"=list(models)))

In [9]:
cl = makeCluster(4)
registerDoParallel(cl)
 
results = foreach(setup = setups, 
        .combine = bind_rows,
        .packages = (.packages()))  %dopar%  
loop_test(setup) 

stopCluster(cl)

In [10]:
results %>% data.frame

,model,selection_method,true_hte_error,true_value,DGP,n_train,n_test,n_folds,rep
1,ranger~5~variance~1,c_benefit,0.374957669731415,0.036965459290384,scenario_1,1000,2000,2,1
2,enet~0.5~7.38905609893065,gain,0.202253700314934,0.036965459290384,scenario_1,1000,2000,2,1
3,enet~0.5~2.71828182845905,match_decision,0.170519247300098,0.036965459290384,scenario_1,1000,2000,2,1
4,enet~0.5~0.00673794699908547,match_mse,0.0575876107623068,0.036965459290384,scenario_1,1000,2000,2,1
5,ranger~5~variance~1,prediction_error,0.374957669731415,0.036965459290384,scenario_1,1000,2000,2,1
6,ranger~3~variance~1,qini,0.262957023344298,0.036965459290384,scenario_1,1000,2000,2,1
7,enet~0.5~2.71828182845905,trans_decision,0.170519247300098,0.036965459290384,scenario_1,1000,2000,2,1
8,enet~0.5~0.00673794699908547,trans_mse,0.0575876107623068,0.036965459290384,scenario_1,1000,2000,2,1
9,enet~0.5~2.71828182845905,value,0.170519247300098,0.036965459290384,scenario_1,1000,2000,2,1
10,enet~0.5~7.38905609893065,value_auc,0.202253700314934,0.036965459290384,scenario_1,1000,2000,2,1
